In [ ]:
import timm
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import torch


#### Basic CNN

In [ ]:
class SimpleCNN1(nn.Module):
    def __init__(self):
      super(SimpleCNN1, self).__init__()

      self.conv_layers = nn.Sequential(
          nn.Conv2d(3, 32, kernel_size=3, padding = 1),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(32, 64, kernel_size=3, padding = 1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(64, 128, kernel_size=3, padding = 1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(128, 256, kernel_size=3, padding = 1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
      )

      self.gap = nn.AdaptiveAvgPool2d((1, 1))
      self.fc_layers = nn.Sequential(
          nn.Flatten(),
          nn.Linear(256, 512),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(512, 1024),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(1024, 10)
      )

    def forward(self, x):
      x = self.conv_layers(x)
      x = self.gap(x)
      x = self.fc_layers(x)
      return x


model = SimpleCNN1()

#### Hybrid CNN-Pretrained Model

In [ ]:
class HybridCNN(nn.Module):
  def __init__(self, num_classes=5):
    super(HybridCNN, self).__init__()

    self.feature_extraction = nn.Sequential(
          nn.Conv2d(3, 32, kernel_size = 5, padding = 2),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),

          nn.Conv2d(32, 64, kernel_size = 3, padding = 2),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),

          nn.Conv2d(64, 128, kernel_size = 5, padding = 2),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),

          nn.Conv2d(128, 256, kernel_size = 5, padding = 2),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),
    )

    self. 
    for param in self.global_cnn.parameters():
        param.requires_grad = False

    for param in self.global_cnn.blocks[6].parameters():
        param.requires_grad = True


    in_features = self.global_cnn.get_classifier().in_features
    self.global_cnn.classifier = nn.Identity()
    self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

    self.fusion_head = nn.Sequential(
            nn.Linear(in_features + 256 , 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

  def forward(self, x):
    local_features = self.feature_extraction(x)
    local_feat = self.global_pool(local_features)
    local_feat = torch.flatten(local_feat, 1)

    global_features = self.global_cnn(x)

    fused = torch.cat((local_feat, global_features), dim=1)

    out = self.fusion_head(fused)
    return out

#### Double Fine Tune Hybrid Single Pretrained

In [ ]:
import torch
import torch.nn as nn
import timm

class HybridFineTune(nn.Module):
    def __init__(self, num_classes, convnext_weight_path=None, freeze_backbone=True, dropout=0.3):
        super(HybridFineTune, self).__init__()

        # === EfficientNet backbone (fine-grained features) ===
        self.eff = timm.create_model('efficientnet_b0', pretrained=False, num_classes=0)
        eff_features = self.eff.num_features

        # === ConvNeXt backbone (general features) ===
        self.convnext = timm.create_model('convnext_tiny', pretrained=False, num_classes=0)
        if convnext_weight_path is not None:
            print(f"Loading ConvNeXt weights from {convnext_weight_path} ...")
            state_dict = torch.load(convnext_weight_path, map_location='cpu')
            self.convnext.load_state_dict(state_dict, strict=False)
        conv_features = self.convnext.num_features

        # === Freeze backbone (optional) ===
        if freeze_backbone:
            for p in self.eff.parameters():
                p.requires_grad = False
            for p in self.convnext.parameters():
                p.requires_grad = False

        # === Fusion layer ===
        total_features = eff_features + conv_features
        self.classifier = nn.Sequential(
            nn.Linear(total_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_out = self.eff(x)
        conv_out = self.convnext(x)

        # Concatenate kedua feature
        combined = torch.cat((eff_out, conv_out), dim=1)
        out = self.classifier(combined)
        return out


#### Siamese Network

In [ ]:
####### Datase Pair Loader #########

import torch
from torch.utils.data import Dataset
from PIL import Image
import random
import os

class ImagePairDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.transform = transform
        self.pairs = []
        self.labels = []
        classes = os.listdir(base_dir)
        
        for c in classes:
            files = os.listdir(os.path.join(base_dir, c))
            # Positive pairs
            for i in range(len(files)-1):
                self.pairs.append((os.path.join(base_dir, c, files[i]), os.path.join(base_dir, c, files[i+1])))
                self.labels.append(1)
            # Negative pairs
            other_class = random.choice([x for x in classes if x != c])
            neg_file = random.choice(os.listdir(os.path.join(base_dir, other_class)))
            self.pairs.append((os.path.join(base_dir, c, files[0]), os.path.join(base_dir, other_class, neg_file)))
            self.labels.append(0)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        path1, path2 = self.pairs[idx]
        img1, img2 = Image.open(path1).convert("RGB"), Image.open(path2).convert("RGB")
        if self.transform:
            img1, img2 = self.transform(img1), self.transform(img2)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return img1, img2, label


In [ ]:
import torch.nn as nn
import torchvision.models as models

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        base_model = models.resnet18(weights='IMAGENET1K_V1')
        base_model.fc = nn.Identity()  # hapus layer klasifikasi
        self.backbone = base_model

        # Binary classifier di atas perbedaan embedding
        self.classifier = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        return self.backbone(x)

    def forward(self, img1, img2):
        feat1 = self.forward_once(img1)
        feat2 = self.forward_once(img2)
        diff = torch.abs(feat1 - feat2)
        out = self.classifier(diff)
        return out


In [ ]:
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(5):
    model.train()
    total_loss = 0
    for img1, img2, label in tqdm(train_loader):
        img1, img2, label = img1.to(device), img2.to(device), label.to(device).unsqueeze(1)

        optimizer.zero_grad()
        output = model(img1, img2)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")
